In [1]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("Airline Performance").getOrCreate()

In [3]:
#read records and make sql dataframe
flights = spark.read.option("header",True).csv('hdfs:///airline_performance/airline_2m.csv')
flights.createOrReplaceTempView("flights")

2022-04-12 15:50:16,191 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
#find airports as vertices in graph
airports = spark.sql("select Origin as id from flights union select Dest from flights")

In [5]:
#(origin airport, dest airport, airline) key stats as edges
stats = spark.sql("select Origin as src, Dest as dst, Reporting_Airline as airline, Year, Quarter, Month, DayofMonth, DayOfWeek, Origin, OriginStateName, Dest, DestStateName, DepDelay, ArrDelay, CRSDepTime, DepTime, Cancelled, CRSElapsedTime, ActualElapsedTime, Distance from flights")

In [6]:
from graphframes import *

In [7]:
g = GraphFrame(airports, stats)

In [11]:
result = g.edges.filter("Distance > 800")

In [ ]:
airports.coalesce(1).write.csv("hdfs:///airline_performance/dest")